This is the global parameters that are set. This includes the subject and the file directories. The PAR.subjects value needs to be changed to match which subject you want to analyze


In [ ]:
addpath(genpath(pwd));


fprintf('\r%s\n',repmat(sprintf('-'),1,30))
fprintf('%-40s\n','Set PAR')


clear
global PAR
PAR=[];


PAR.SPM_path=spm('Dir');
addpath(PAR.SPM_path);

% This file sets up various things specific to this
% analysis, and stores them in the global variable PAR,
% which is used by the other batch files.
% You don't have to do it this way of course, I just
% found it easier



%%%%%%%%%%%%%%%%%%%%%
%                   %
%   GENERAL PREFS   %
%                   %
%%%%%%%%%%%%%%%%%%%%%%
% Where the subjects' data directories are stored

PAR.batchcode_which= mfilename('fullpath');
PAR.batchcode_which=fileparts(PAR.batchcode_which);
addpath(PAR.batchcode_which);
old_pwd=pwd;
cd(pwd);
cd ../
data_root=pwd;
cd(old_pwd);


PAR.root=data_root;

% Subjects' directories
% Enter the correct sbuject directory
PAR.subjects = {'sub1' };
PAR.nsubs = length(PAR.subjects);



% Anatomical directory name
PAR.structfilter='Anatomy';


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Get the anatomical image directories automatically
for sb=1:PAR.nsubs
    tmp=dir(fullfile(PAR.root,PAR.subjects{sb},['*' PAR.structfilter '*']));
    if size(tmp,1)==0
        sprintf('Can not find the anatomical directory for subject\n')
        sprintf('%s: \n', char(PAR.subjects{sb}))
        error('Can not find the anatomical directory for subject');
    end
    if size(tmp,1)>1
        sprintf('More than 1 anatomical directories for subject: %s are found here!\n',char(PAR.subjects{sb}))
        error('More than 1 anatomical directories are found')
    end
    PAR.structdir{sb}=fullfile(PAR.root,PAR.subjects{sb},spm_str_manip(char(tmp(1).name),'d'));
end
%prefixes for filenames of structural 3D images, supposed to be the same for every subj.
PAR.structprefs = 'Anatomy';
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Getting condition directories
PAR.confilters={'ASL_S1'}; %filter for folder name and filenames of functional images
PAR.ncond=length(PAR.confilters);
PAR.funcimgfilters={'ASLREST'};

% The condition names are assumed same for different sessions

for sb=1:PAR.nsubs
    for c=1:PAR.ncond
        tmp=dir(fullfile(PAR.root,PAR.subjects{sb},['*' PAR.confilters{c} '*']));

        if size(tmp,1)==0
            sprintf('Can not find the condition directory for subject\n')
            sprintf('%s: \n', char(PAR.subjects{sb}))
            error('Can not find the condition directory for subject');
        end

        if size(tmp,1)>1
            sprintf('Panic! subject %s has more than 1 directories!\n', [PAR.subjects{sb}])
            error('Panic! condition has more than 1 directories!')
            %return;
        end
        PAR.condirs{sb,c}=fullfile(PAR.root,PAR.subjects{sb},spm_str_manip(char(tmp(1).name),'d'));
    end
end

% Smoothing kernel size
PAR.FWHM = [6];

% % TR for each subject.  As one experiment was carried out in one Hospital (with one machine)
% % and the other in another hospital (different machine), TRs are slightly different
% %PAR.TRs = [2.4696 2];
PAR.TRs = ones(1,PAR.nsubs)*6;
% PAR.mp='no';

%
PAR.mp='no';
%
PAR.groupdir = ['STAT'];


PAR.subtractiontype=0;
PAR.glcbffile=['globalsg_' num2str(PAR.subtractiontype) '.txt'];
PAR.img4analysis='cbf'; % or 'Perf'
PAR.ana_dir = ['glm_' PAR.img4analysis];
PAR.Filter='cbf_0_sr';





In [ ]:
PAR.subjects

Reset orientation

In [ ]:
% batch reset T1 orientation

fprintf('\r%-40s\n','batch reset orientation ')
%%%%% the following codes are changed from batch_smooth

% dirnames,
% get the subdirectories in the main directory
for sb = 1:PAR.nsubs % for each subject
    %go get the sessions
    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb} );
    fprintf('\r%-40s  %30s',str,' ')
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...reseting')  %-#
    %now get all images to reset orientation
    %prepare directory
    P=[];
    for c=1:PAR.ncond
        % get files in this directory
        P=spm_select('FPList', char(PAR.condirs{sb,c}), ['^' PAR.funcimgfilters{c} '.*\.nii$']);
%         ASLtbx_resetimgorgASL(P);
        ASLtbx_resetimgorg(P);
    end

    P=spm_select('FPList', char(PAR.structdir{sb}), ['^' PAR.structprefs '.*\.nii$']);
    ASLtbx_resetimgorg(P);
    
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...done')  %-#
end
fprintf('\n');



In [ ]:
img=load_nii(P)

In [ ]:
imatlab_export_fig('print-png')
imagesc(img.img(:,:,40))

Realign the images

In [ ]:

disp('We are now realigning raw EPI images for all subjects, just wait....');
% load spm defaults
global defaults;
spm_defaults;

% Get realignment defaults
defs = defaults.realign;

% Flags to pass to routine to calculate realignment parameters
% (spm_realign)

%as (possibly) seen at spm_realign_ui,
% -fwhm = 5 for fMRI
% -rtm = 0 for fMRI
% for this particular data set, we did not perform a second realignment to the mean
% the coregistration between the reference control and label volume is also omitted
reaFlags = struct(...
    'quality', defs.estimate.quality,...  % estimation quality
    'fwhm', 5,...                         % smooth before calculation
    'rtm', 1,...                          % whether to realign to mean
    'PW',''...                            %
    );

% Flags to pass to routine to create resliced images
% (spm_reslice)
resFlags = struct(...
    'interp', 1,...                       % trilinear interpolation
    'wrap', defs.write.wrap,...           % wrapping info (ignore...)
    'mask', defs.write.mask,...           % masking (see spm_reslice)
    'which',2,...                         % write reslice time series for later use
    'mean',1);                            % do write mean image


% dirnames,
% get the subdirectories in the main directory
for sb =1:PAR.nsubs % for each subject
    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb} );
    fprintf('\r%-40s  %30s',str,' ')
    P=[];
    for c=1:PAR.ncond
        % get files in this directory
        Ptmp=spm_select('ExtFPList',PAR.condirs{sb,c},['^' PAR.funcimgfilters{c} '\w*\.nii$'],1:400);
        P=strvcat(P,Ptmp);
    end
    spm_realign_asl(P, reaFlags);
    % Run reslice
    spm_reslice(P, resFlags);

end


Coregister the images

In [ ]:

disp('Coregistration between structural images and the functional images for all subjects, it takes a while....');
global defaults;
spm_defaults;
flags = defaults.coreg;

% dirnames,
% get the subdirectories in the main directory
for s = 1:length(PAR.subjects) % for each subject
    %take the dir where the mean image (reslice) is stored (only first condition)
    sprintf('\nNow coregister %s''s data\n',char(PAR.subjects{s}))
    dir_fun = PAR.condirs{s,1};
    %take the structural directory
    dir_anat = PAR.structdir{s};
    % get mean in this directory
    %target - Tar(G)et image, NEVER CHANGED
    %source - Source image, transformed to match PG
    %otherimgs - (O)ther images, originally realigned to PF and transformed again to PF

    target = spm_select('FPList',PAR.structdir{s},['^' PAR.structprefs '\w*\.nii$']);
    source = spm_select('FPList', dir_fun,   ['^mean' PAR.funcimgfilters{1} '\w*\.nii$']);
    PO=[];
    for c=1:PAR.ncond
        % get files in this directory
       % Ptmp=spm_get('files', PAR.condirs{s,c}, ['r*img']);
        Ptmp=spm_select('EXTFPList', char(PAR.condirs{s,c}), ['^r' PAR.funcimgfilters{c} '.*\.nii'], 1:400);
        PO=strvcat(PO,Ptmp);
    end
    if isempty(PO) | PO=='/'
        PO=source;
    else
        PO = strvcat(source,PO);
    end
    otherimgs = PO;
    ASLtbx_spmcoreg(target, source, PO);
end


Filter the images

In [ ]:
% Get subject etc parameters
disp('Smoothing the realigned functional images, it is quick....');
org_pwd=pwd;
% dirnames,
% get the subdirectories in the main directory
for sb = 1:PAR.nsubs % for each subject

    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb});
    fprintf('\r%-40s  %30s',str,' ')
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...smoothing')  %-#

    for c=1:PAR.ncond
        meanimg=spm_select('FPList', PAR.condirs{sb,c}, ['^mean' PAR.funcimgfilters{c} '\w*\.nii$']);
        ASLtbx_createbrainmask(meanimg);
        rimgs=spm_select('EXTFPList', char(PAR.condirs{sb,c}), ['^r' PAR.funcimgfilters{c} '.*nii'], 1:1000);
        maskimg=spm_select('FPList', PAR.condirs{sb,c}, ['^brainmask\.nii']);
        ASLtbx_asltemporalfiltering(rimgs, maskimg);
    end
end

Smooth the images

In [ ]:
% Get subject etc parameters
disp('Smoothing the realigned functional images, it is quick....');
org_pwd=pwd;
% dirnames,
% get the subdirectories in the main directory
for sb = 1:PAR.nsubs % for each subject

    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb});
    fprintf('\r%-40s  %30s',str,' ')
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...smoothing')  %-#

    for c=1:PAR.ncond
        %P=[];
        %dir_func = fullfile(PAR.root, PAR.subjects{s},PAR.sesses{ses},PAR.condirs{c});
        
              
        fltimgs=spm_select('FPList', PAR.condirs{sb,c}, '^ASLflt.*nii'); 
        

        if isempty(fltimgs), fprintf('You didn''t selecte any images!\n'); return;end;
        if size(fltimgs,1)==1
            [pth,nam,ext,num] = spm_fileparts(fltimgs);
            fltimgs=fullfile(pth, [nam ext]);
        end
        [pth,nam,ext,num] = spm_fileparts(fltimgs(1,:));
         ASLtbx_smoothing(fltimgs, PAR.FWHM);
        
       
    end

end
cd(org_pwd);

Create masks for perfusion signals

In [ ]:

for sb = 1:PAR.nsubs % for each subject
    clear P;
    for c=1:PAR.ncond
        % creating a mask image for removing background
        dir_fun = PAR.condirs{sb,1};
        PF=spm_select('ExtFPList',PAR.condirs{sb,c},['^mean' PAR.funcimgfilters{c} '\w*\.nii$'],1:200);
        if size(PF,1)<1
            fprintf('No mean images for subject %s!\n',PAR.subjects{sb});
            continue;
        end
        PF=PF(1,:);
        vm=spm_vol(PF);
        dat=spm_read_vols(vm);
        mask=dat>0.2*max(dat(:));
        vo=vm;
        [path,name,ext]=fileparts(PF);
        vo.fname=fullfile(path,'brainmask_nat.nii');
        vo=spm_write_vol(vo,mask);
    end
end

Perfusion calculation

In [ ]:
for sb = 1:PAR.nsubs % for each subject

    sprintf('Calculate perfusion and CBF signals for subject #%g ... %g subjects left...',sb,length(PAR.subjects)-sb)
    for c=1:PAR.ncond

        P=[];
        %ptmp=spm_get('files', PAR.condirs{s,1}, [PAR.subjects{s} '*' PAR.confilters{1} '*img']);
        %P=ptmp(1,:);
        % creating a mask image for removing background
        maskimg = spm_select('FPList', PAR.condirs{sb,c},    ['^brainmask_nat\w*\.nii$']);
        Filename =  spm_select('ExtFPList', PAR.condirs{sb,c}, ['^sASLflt.*\.nii'], 1:1000 );
        %        spm_perf_subtract(Filename,FirstimageType, SubtractionType,...
        %           SubtractionOrder,Flag,Timeshift,AslType,MagType,Labeltime,Delaytime,Slicetime,M0img,M0seg,maskimg)
        
                %asl_perf_subtract(Filename,FirstimageType, SubtractionType,...
        %   SubtractionOrder,Flag,
        %Timeshift,AslType,labeff,MagType,
        %   Labeltime,Delaytime,Slicetime,TE,M0img,M0seg,maskimg)
        
        % These are the values that impact the actual calculation of the
        % perfusion 
        FirstImage=0; % What is the first image? Control or label
        SubtractionType = 0; 
        SubtractionOrder=0;
        % Flag is more complex, these settings ask what outputs you would
        % like to save and create
        % Flag = [1 1 1 0 0 1 0]; % For PASL
        Flag = [1 1 1 0 0 1 0 1 1]; % For pCASL
        Timeshift = 0.5;
        AslType = 1; % Is the ASL PASL, CASL, or pCASL? 0=PASL, 1=CASL/pCASL 
        labeff=0.85; % What is the label efficiency? 
        MagType = 1; % What field strength? 0 = 1.5T, 1=3T
        Labeltime= 1.48; % What is the labeling time?
        DelayTime = 1.5; % What is the post labeling delay? 
        SliceTime = 33.5; % To correct for slice timing delays, what is the slice time (only for 2D acquisitions
        TE=11; % What is the TE for the acquisition?
   
        
        asl_perf_subtract(Filename, FirstImage, SubtractionType, ...
            SubtractionOrder, Flag, Timeshift,     AslType,     labeff, MagType,...
            Labeltime, DelayTime, SliceTime, TE, [],[],maskimg);
        % labeling time=1.48s, post labeling delay=1.5 s, slice time=33.5
        % msec, TE=11 msec, no explicit M0 image, no mask for extracting M0
        % value (like M0 white matter etc) provided.
    end
end

In [ ]:
CBF=load_nii('../sub5/ASL_S1/meanCBF_0_sASLflt_rASLREST.nii')

In [ ]:
imagesc(CBF.img(:,:,9))